In [20]:
from fishsense_api_sdk.client import Client
from fishsense_api_sdk.models.image import Image
from pathlib import Path
import asyncio
from typing import List
import numpy as np
import rawpy
import matplotlib.pyplot as plt
from skimage.util import img_as_float, img_as_ubyte
import cv2
import math
from tqdm.notebook import tqdm
from skimage.exposure import adjust_gamma, equalize_adapthist
from synology_api.filestation import FileStation
from fishsense_data_processing_workflow_worker.config import settings
import os
from urllib.parse import urlparse
from fishsense_api_sdk.models.data_source import DataSource
from fishsense_core.image.raw_image import RawImage
from fishsense_core.image.rectified_image import RectifiedImage

In [21]:
# os.getcwd()
# os.chdir('..')

# os.getcwd()

In [22]:
DIVE_ID = 383

In [23]:
DATA_FOLDER = (Path("../data") / "REEF" / "data").absolute()
OUTPUT_FOLDER = (Path("../output") / "preprocess_groups_jpeg").absolute()

DATA_FOLDER.mkdir(parents=True, exist_ok=True)
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

DATA_FOLDER.exists(), OUTPUT_FOLDER.exists()

(True, True)

In [24]:
parsed_nas_url = urlparse(settings.e4e_nas.url)

filestation = FileStation(parsed_nas_url.hostname, parsed_nas_url.port, settings.e4e_nas.username, settings.e4e_nas.password, secure=True, cert_verify=False)

In [25]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as client:
    dive = await client.dives.get(dive_id=DIVE_ID)

dive

Dive(id=383, name='062624_FiveSeas_FSL03', path='2025-02-10 REEF Data Dump SMILE 6/CubaTrip2024_FSL03/062624_FiveSeas_FSL03', dive_datetime=datetime.datetime(2024, 6, 26, 7, 52, 45, tzinfo=TzInfo(0)), priority=<Priority.LOW: 'LOW'>, flip_dive_slate=None, camera_id=3, dive_slate_id=None)

In [26]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as client:
    camera = await client.cameras.get(dive.camera_id)

camera

Camera(id=3, serial_number='BJ6C85528', name='FSL-03')

In [27]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as client:
    camera_intrinsics = await client.cameras.get_intrinsics(dive.camera_id)

camera_intrinsics

In [28]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as client:
    clusters = await client.images.get_clusters(DIVE_ID, DataSource.PREDICTION.value)

len(clusters), len(clusters[0].image_ids), clusters[0].image_ids[0]

(13, 5, 111930)

In [29]:
cluster_images: List[List[Image]] = []

async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as client:
    for cluster in clusters:
        cluster_images.append(await asyncio.gather(*[client.images.get(image_id=i) for i in cluster.image_ids]))

sum(len(c) for c in cluster_images), len(cluster_images), len(cluster_images[0]), cluster_images[0][0]

(40,
 13,
 5,
 Image(id=111930, path='2025-02-10 REEF Data Dump SMILE 6/CubaTrip2024_FSL03/062624_FiveSeas_FSL03/P6260581.ORF', taken_datetime=datetime.datetime(2024, 6, 26, 7, 20, 29, tzinfo=TzInfo(0)), checksum='c5cef715ad648236aac9c06f762d2ead', is_canonical=True, dive_id=383, camera_id=3))

In [30]:
for cluster in tqdm(cluster_images):
    for idx, image in enumerate(tqdm(cluster)):
        image_path = DATA_FOLDER / image.path
        target_path = OUTPUT_FOLDER / f"{image.checksum}.JPG"

        source_nas_path = f"/fishsense_data/REEF/data/{image.path}"
        filestation.get_file(source_nas_path, "download", dest_path=str(image_path.parent))

        image = RectifiedImage(RawImage(image_path), camera_intrinsics)
        img = image.data

        height, width = img.shape[:2]

        # Define text properties
        text = f"{idx + 1}/{len(cluster)}"
        org = (width - 350, height - 75) # Bottom-right corner of the text
        fontFace = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 5
        color = (0, 0, 255) # Red color (BGR)
        thickness = 10
        lineType = cv2.LINE_AA

        # Put the text on the image
        cv2.putText(img, text, org, fontFace, fontScale, color, thickness, lineType)

        cv2.imwrite(target_path.as_posix(), img)

        target_nas_path = f"/fishsense_process_work/preprocess_groups_jpeg"
        filestation.upload_file(target_nas_path, str(target_path), overwrite=True)

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

<MultipartEncoder: {'path': '/fishsense_process_work/preprocess_groups_jpeg', 'create_parents': 'true', 'overwrite': 'true', 'size': '3421934', 'files': ('c5cef715ad648236aac9c06f762d2ead.JPG', <_io.BufferedReader name='/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/output/preprocess_groups_jpeg/c5cef715ad648236aac9c06f762d2ead.JPG'>, 'image/jpeg')}>

















Upload Progress: 100%|██████████| 3.26M/3.26M [00:03<00:00, 903kB/s] 


<MultipartEncoder: {'path': '/fishsense_process_work/preprocess_groups_jpeg', 'create_parents': 'true', 'overwrite': 'true', 'size': '3364075', 'files': ('c7e168db6dd10533af694a1a637409b7.JPG', <_io.BufferedReader name='/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/output/preprocess_groups_jpeg/c7e168db6dd10533af694a1a637409b7.JPG'>, 'image/jpeg')}>


...

















Upload Progress: 100%|██████████| 3.30M/3.30M [00:04<00:00, 735kB/s]


<MultipartEncoder: {'path': '/fishsense_process_work/preprocess_groups_jpeg', 'create_parents': 'true', 'overwrite': 'true', 'size': '3411113', 'files': ('4711763b48368af835dc397c61411dab.JPG', <_io.BufferedReader name='/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/output/preprocess_groups_jpeg/4711763b48368af835dc397c61411dab.JPG'>, 'image/jpeg')}>























Upload Progress: 100%|██████████| 3.25M/3.25M [00:04<00:00, 822kB/s] 


<MultipartEncoder: {'path': '/fishsense_process_work/preprocess_groups_jpeg', 'create_parents': 'true', 'overwrite': 'true', 'size': '3478284', 'files': ('a16c10d11e3ea788f966800c77af1501.JPG', <_io.BufferedReader name='/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/output/preprocess_groups_jpeg/a16c10d11e3ea788f966800c77af1501.JPG'>, 'image/jpeg')}>


















Upload Progress: 100%|██████████| 3.32M/3.32M [00:03<00:00, 924kB/s] 
